In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!sudo apt install git-lfs
!git lfs version
!rm -rf ml
!mkdir ml
!cd ml
!git init
!git lfs install
!git clone https://github.com/kokoichi206/ml.git
!git fetch
!cd ..
!ls -la ml/akimoto/
!mv ml/akimoto/saka.zip saka.zip
!mv ml/akimoto/akb.zip akb.zip

!rm -rf akb_pre
!rm -rf saka_pre

!ls
!yes | unzip akb.zip > /dev/null
!yes | unzip saka.zip > /dev/null

!ls akb_pre | wc
!ls saka_pre | wc

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  git-lfs
0 upgraded, 1 newly installed, 0 to remove and 37 not upgraded.
Need to get 2,129 kB of archives.
After this operation, 7,662 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 git-lfs amd64 2.3.4-1 [2,129 kB]
Fetched 2,129 kB in 0s (20.3 MB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package git-lfs.
(Reading database ... 155222 files and directories cu

In [14]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt


dataset = keras.preprocessing.image_dataset_from_directory(
    directory="saka_pre", label_mode=None, image_size=(64, 64), batch_size=32,
    shuffle=True
).map(lambda x: x/255.0)

Found 3928 files belonging to 1 classes.


In [15]:
discriminator = keras.Sequential(
    [
        keras.Input(shape=(64,64,3)),
        layers.Conv2D(64, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(0.2),
        layers.Conv2D(128, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(0.2),
        layers.Conv2D(128, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(0.2),
        layers.Flatten(),
        layers.Dropout(0.2),
        layers.Dense(1, activation="sigmoid"),
    ]
)

discriminator.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 32, 32, 64)        3136      
                                                                 
 leaky_re_lu_6 (LeakyReLU)   (None, 32, 32, 64)        0         
                                                                 
 conv2d_5 (Conv2D)           (None, 16, 16, 128)       131200    
                                                                 
 leaky_re_lu_7 (LeakyReLU)   (None, 16, 16, 128)       0         
                                                                 
 conv2d_6 (Conv2D)           (None, 8, 8, 128)         262272    
                                                                 
 leaky_re_lu_8 (LeakyReLU)   (None, 8, 8, 128)         0         
                                                                 
 flatten_1 (Flatten)         (None, 8192)             

In [16]:
latent_dim = 128
generator = keras.Sequential(
    [
        layers.Input(shape=(latent_dim, )),
        layers.Dense(8*8*128),
        layers.Reshape((8, 8, 128)),
        layers.Conv2DTranspose(128, kernel_size=4, strides=2, padding="same"),  # 16*16
        layers.LeakyReLU(0.2),
        layers.Conv2DTranspose(256, kernel_size=4, strides=2, padding="same"), # 32*32
        layers.LeakyReLU(0.2),
        layers.Conv2DTranspose(512, kernel_size=4, strides=2, padding="same"), # 64*64
        layers.LeakyReLU(0.2),
        layers.Conv2D(3, kernel_size=5, padding="same", activation="sigmoid")
    ]
)

generator.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 8192)              1056768   
                                                                 
 reshape_1 (Reshape)         (None, 8, 8, 128)         0         
                                                                 
 conv2d_transpose_3 (Conv2DT  (None, 16, 16, 128)      262272    
 ranspose)                                                       
                                                                 
 leaky_re_lu_9 (LeakyReLU)   (None, 16, 16, 128)       0         
                                                                 
 conv2d_transpose_4 (Conv2DT  (None, 32, 32, 256)      524544    
 ranspose)                                                       
                                                                 
 leaky_re_lu_10 (LeakyReLU)  (None, 32, 32, 256)      

In [17]:
opt_gen = keras.optimizers.Adam(1e-4)
opt_disc = keras.optimizers.Adam(1e-4)
loss_fn = keras.losses.BinaryCrossentropy()

In [ ]:
for i in range(20):
    print(f"======= i: {i} =======")
    for epoch in range(100*i, 100*(i+1)):
        for idx, (real) in enumerate(tqdm(dataset)):
            batch_size = real.shape[0]
            with tf.GradientTape() as gen_tape:
                random_latent_vectors = tf.random.normal(shape = (batch_size, latent_dim))
                fake = generator(random_latent_vectors)

            # 1epoch に１回写真を保存
            if idx == 0:
                img = keras.preprocessing.image.array_to_img(fake[0])
                img.save("/content/drive/MyDrive/Colab Notebooks/akimoto/gan_generated/epoch_%04d.png" % (epoch))

            ### Train Discriminator: max log(D(x)) + log(1 - D(G(z)))
            with tf.GradientTape() as disc_tape:
                loss_disc_real = loss_fn(tf.ones((batch_size, 1)), discriminator(real))
                loss_disc_fake = loss_fn(tf.zeros((batch_size, 1)), discriminator(fake))
                loss_disc = (loss_disc_real + loss_disc_fake)/2

            grads = disc_tape.gradient(loss_disc, discriminator.trainable_weights)
            opt_disc.apply_gradients(
                zip(grads, discriminator.trainable_weights)
            )

            ### Train Generator: min log(1 - D(G(z))) <-> max log(D(G(z))
            with tf.GradientTape() as gen_tape:
                fake = generator(random_latent_vectors)
                output = discriminator(fake)
                loss_gen = loss_fn(tf.ones(batch_size, 1), output)

            grads = gen_tape.gradient(loss_gen, generator.trainable_weights)
            opt_gen.apply_gradients(zip(grads, generator.trainable_weights))

    # 途中でcolabが止まった時の対策に、定期的にparameterを保存
    gen_string = generator.to_json()
    with open("/content/drive/MyDrive/Colab Notebooks/akimoto/gan_models/gen_model.json", mode='w') as f:
        f.write(gen_string)
    generator.save_weights("/content/drive/MyDrive/Colab Notebooks/akimoto/gan_models/gen_weights.hdf5")

    disc_string = discriminator.to_json()
    with open("/content/drive/MyDrive/Colab Notebooks/akimoto/gan_models/disc_model.json", mode='w') as f:
        f.write(disc_string)
    discriminator.save_weights("/content/drive/MyDrive/Colab Notebooks/akimoto/gan_models/disc_weights.hdf5")



  2%|▏         | 2/123 [00:00<00:24,  5.02it/s]

In [8]:
gen_string = generator.to_json()
with open("/content/drive/MyDrive/Colab Notebooks/akimoto/gan_models/gen_model.json", mode='w') as f:
    f.write(gen_string)
generator.save_weights("/content/drive/MyDrive/Colab Notebooks/akimoto/gan_models/gen_weights.hdf5")

disc_string = discriminator.to_json()
with open("/content/drive/MyDrive/Colab Notebooks/akimoto/gan_models/disc_model.json", mode='w') as f:
    f.write(disc_string)
discriminator.save_weights("/content/drive/MyDrive/Colab Notebooks/akimoto/gan_models/disc_weights.hdf5")
